In [1]:
import numpy as np 
import pandas as pd

In [2]:
from google.colab import drive 
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/Datasets /archive (23).zip'

In [4]:
df = pd.read_csv('/content/Admission_Predict.csv')

In [5]:
pd.set_option('display.max_rows',400)

In [6]:
np.random.seed(10)

n_sample = df.shape[0]
shuffled_indices = np.random.permutation(n_sample)
#print(shuffled_indices)

In [7]:
features = ['GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Chance of Admit ']

x = np.array(df[features])
y = np.array(df['Research'])

In [8]:
n_val =int( 0.1 * df.shape[0])
print(n_val)

train_indices = shuffled_indices[n_val:]
test_indices = shuffled_indices[:n_val]

x_train = x[train_indices]
x_test  = x[test_indices]

y_train = y[train_indices]
y_test  = y[test_indices]

40


In [9]:
x_train

array([[332.  , 116.  ,   5.  , ...,   5.  ,   9.28,   0.94],
       [326.  , 112.  ,   4.  , ...,   3.5 ,   9.12,   0.84],
       [327.  , 114.  ,   3.  , ...,   3.  ,   9.02,   0.61],
       ...,
       [314.  , 105.  ,   3.  , ...,   2.5 ,   8.3 ,   0.54],
       [300.  , 100.  ,   3.  , ...,   3.  ,   8.66,   0.64],
       [313.  , 102.  ,   3.  , ...,   2.5 ,   8.68,   0.71]])

In [10]:
x_train = (x_train - x_train.mean(axis = 0))/x_train.std(axis = 0)
x_test = (x_test - x_test.mean(axis = 0))/x_test.std(axis = 0)

In [11]:
x_train = x_train.reshape(7,-1)
y_train = y_train.reshape(1,-1)
x_test = x_test.reshape(7,-1)
y_test = y_test.reshape(1,-1)

In [20]:
class NN:

  def __init__(self,input_size,h1_size,output_size):
    #-----Inputs are assigned to the model----#
    self.input_size = input_size
    self.h1_size = h1_size 
    self.output_size = output_size 
    
    #----Initiailze the weights----#
    self.W1 = np.random.uniform(low = -1,high = 1, size = (self.h1_size,self.input_size))
    self.b1 = np.zeros([self.h1_size,1])
    self.W2 = np.random.uniform(low = -1,high = 1, size = (self.output_size,self.h1_size))
    self.b2 = np.zeros([self.output_size,1])

  def forward(self,x):
    self.Z1 = np.dot(self.W1,x) + self.b1
    self.A1 = self.Tanh(self.Z1)

    self.Z2 = np.dot(self.W2,self.A1) + self.b2
    self.A2 = self.Sigmoid(self.Z2)
    return self.A2

  def NLLloss(self,output,y):
    loss = -np.sum((np.dot(y,np.log(self.A2).T) + np.dot(1-y,np.log(1-self.A2).T))/x.shape[1])
    return loss

  def backward(self,x,y):
    self.dA2 = (self.A2-y)/np.multiply(self.A2,1-self.A2)
    self.dZ2 = self.dA2*self.Sigmoid(self.Z2,derivative = True)
    self.dW2 = np.dot(self.dZ2,self.A1.T)/x.shape[1]
    self.db2 = np.sum(self.dZ2, axis = 1 , keepdims = True)/x.shape[1]

    self.dA1 = np.dot(self.W2.T,self.dZ2)
    self.dZ1 = self.dA1*self.Tanh(self.Z1, derivative = True)
    self.dW1 = np.dot(self.dZ1,x.T)/x.shape[1]
    self.db1 = np.sum(self.dZ1 , axis = 1 , keepdims = True)/x.shape[1]

  #-----Define various activation functions-----#
  def Sigmoid(self,x,derivative = False):
    if derivative == False:
      x = 1/(1 + np.exp(-x))
      return x
    else:
      y = 1/(1 + np.exp(-x))
      return y*(1-y)

  def Tanh(self,x,derivative = False):
    if derivative == False:
      x = np.tanh(x)
      return x
    else:
      y = np.tanh(x)
      return 1-y**2

In [28]:
model = NN(input_size=7,h1_size=64,output_size=1)

In [53]:
outputs_train = model.forward(x_train)
loss = model.NLLloss(outputs,y_train)
model.backward(x_train,y_train)

In [56]:
#------Update the parameters using Gradient Descent------#
lr = 1e-1
num_epochs = 500

for epochs in range(num_epochs):
  outputs = model.forward(x_train)
  loss = model.NLLloss(outputs,y_train)
  model.backward(x_train,y_train)
  #------Update the parameters------#
  model.W1 -= lr*model.dW1
  model.b1 -= lr*model.db1
  model.W2 -= lr*model.dW2
  model.b2 -= lr*model.db2
  #-----Print the loss-----#
  print("[{}/{}] Loss: {}".format(epochs+1,num_epochs,loss))

[1/500] Loss: 3.6351849315180234
[2/500] Loss: 3.6339057973158626
[3/500] Loss: 3.6326273215869915
[4/500] Loss: 3.6313495039434796
[5/500] Loss: 3.630072343997607
[6/500] Loss: 3.628795841361852
[7/500] Loss: 3.6275199956489033
[8/500] Loss: 3.6262448064716466
[9/500] Loss: 3.624970273443178
[10/500] Loss: 3.623696396176793
[11/500] Loss: 3.6224231742859936
[12/500] Loss: 3.6211506073844864
[13/500] Loss: 3.6198786950861837
[14/500] Loss: 3.6186074370051946
[15/500] Loss: 3.6173368327558424
[16/500] Loss: 3.616066881952645
[17/500] Loss: 3.6147975842103324
[18/500] Loss: 3.613528939143832
[19/500] Loss: 3.6122609463682784
[20/500] Loss: 3.610993605499013
[21/500] Loss: 3.6097269161515726
[22/500] Loss: 3.6084608779417087
[23/500] Loss: 3.6071954904853643
[24/500] Loss: 3.6059307533987
[25/500] Loss: 3.60466666629807
[26/500] Loss: 3.6034032288000364
[27/500] Loss: 3.602140440521364
[28/500] Loss: 3.6008783010790233
[29/500] Loss: 3.5996168100901857
[30/500] Loss: 3.5983559671722287
[3

In [39]:
#----Model Accuracy on Test Dataset----#
outputs_test = model.forward(x_test)
loss = model.NLLloss(outputs,y_test)
print(loss)

9.021675127293685


In [40]:
outputs_test

array([[0.205389  , 0.77137199, 0.73165637, 0.46471751, 0.42638496,
        0.00268187, 0.51635608, 0.12607124, 0.56083835, 0.05177593,
        0.41071092, 0.24273937, 0.06309108, 0.02199178, 0.12063565,
        0.00176156, 0.09482397, 0.58843474, 0.02277412, 0.2056326 ,
        0.82797546, 0.82709707, 0.92390927, 0.25769992, 0.80483681,
        0.9949691 , 0.16407127, 0.23773823, 0.67651536, 0.96567751,
        0.04871298, 0.71103873, 0.83025455, 0.84143262, 0.76592178,
        0.74334647, 0.39340191, 0.92991645, 0.9787754 , 0.07628153]])

In [57]:
outputs_train = np.where(outputs_train >= 0.5 , 1 , outputs_train)
outputs_train = np.where(outputs_train < 0.5 , 0 , outputs_train)

In [58]:
(outputs_train == y_train).sum()

360